# Julia for Dynamical Systems and Scientific Machine Learning

### Julia ecosystems of scientific computing

There are many Julia organizations about seemingly every scientific area. Some examples:

- Astrophysics: [JuliaAstro](https://juliaastro.github.io/), [JuliaSpace](https://github.com/JuliaSpace)
- Bio/Chemistry: [JuliaBio](https://biojulia.net/), [Molecular simulations](https://github.com/JuliaMolSim)
- Complex systems, nonlinear dynamics: [JuliaDynamics](https://juliadynamics.github.io/JuliaDynamics/)
- Scientific Machine Learning: [SciML](https://juliadiffeq.org/)
- Solid state: [QuantumOptics](https://qojulia.org/), [JuliaPhysics](https://github.com/JuliaPhysics)
- Economics: [QuantEcon](https://julia.quantecon.org/), [JuliaQuant](https://github.com/JuliaQuant)
- Geosciences/Climate: [JuliaGeo](https://github.com/JuliaGeo), [JuliaEarth](https://github.com/JuliaEarth), [JuliaClimate](https://github.com/JuliaClimate), [CliMA](https://github.com/CliMA)
- Optimization: [JuliaOpt](http://www.juliaopt.org/)
- Machine Learning: [JuliaML](https://juliaml.github.io/), [Flux](https://fluxml.ai/)

among many many others...

For discovering more packages you can take a look at this [curated list](https://github.com/svaksha/Julia.jl) (although some of the topics are a bit outdated), to the [JuliaHub](https://juliahub.com/) or checking out the [JuliaCon's playlists](https://www.youtube.com/user/JuliaLanguage/playlists).



In [ ]:
#Run this while we are working on Atom to save time

using Pkg
Pkg.activate(".")
Pkg.instantiate()
using DifferentialEquations, Plots, Interact, DynamicalSystems, StaticArrays, LinearAlgebra
using DataDrivenDiffEq, ModelingToolkit, DiffEqFlux, DiffEqSensitivity, Flux, Optim, Random

# 1. DifferentialEquations.jl

[DifferentialEquations.jl](https://docs.juliadiffeq.org/latest/) is by far [the best](https://www.stochasticlifestyle.com/comparison-differential-equation-solver-suites-matlab-r-julia-python-c-fortran/) free and open source differential equations solver (not for Julia, for any language). It can solve standard ODEs, Delay-DEs, stochastic DEs, has tools for PDEs, event handling, other multiple features, 100s of solvers, and *even more*.


## Defining and solving some ODEs

The way DifferentialEquations.jl works is quite straightforward:

1. Make your set of ODEs a Julia function `f`
2. Put `f`, an initial state, timespan and a parameter container into an `ODEProblem`.
2. Optionaly choose the solvers and the arguments of the solvers you will use (e.g. tolerances, etc.)
3. Give `f` and auxilary arguments to the function `solve`!

Let's see it in practice by solving the Lorenz system

$$
\begin{aligned}
\dot{x} &= \sigma (y - x) \\
\dot{y} &= x (\rho - z) - y \\
\dot{z} &= xy -\beta z
\end{aligned}
$$

In [ ]:
function lorenz(u,p,t)                        # define the system
    x, y, z = u
    σ, ρ, β = p
    
    ẋ = σ*(y - x)
    ẏ = x*(ρ - z) - y
    ż = x*y - β*z
    
    return [ẋ, ẏ, ż]
end
                                
u₀ = [1.0, 5.0, 10.0]                         # initial conditions
p = [10, 28, 8/3]                             # parameters        
tspan = (0.0, 100.0)                          # timespan
prob = ODEProblem(lorenz, u₀, tspan, p)       # define the problem
sol = solve(prob)                             # solve it
plot(sol, vars = (1, 2, 3))                   # plot the solution

Let's see some other plots

In [ ]:
xyzt = plot(sol)
xy = plot(sol, vars=(1,2))
xz = plot(sol, vars=(1,3))
yz = plot(sol, vars=(2,3))
xyz = plot(sol, vars=(1,2,3))

plot(plot(xyzt,xyz), plot(xy, xz, yz, layout=(1,3)), layout=(2,1))

Another quick example, now providing the equations in-place:

In [ ]:
function lotka_volterra!(du,u,p,t)                        # define the system
    🐇, 🦊 = u
    α, β, γ, δ = p
    
    du[1] = d🐇 =  α*🐇 - β*🐇*🦊
    du[2] = d🦊 = -γ*🦊 + δ*🐇*🦊
end
                                        
ũ₀ = [1.0, 1.0]                                           # initial conditions
timespan = (0.0, 10.0)                                    # timespan
p̃ = [1.5, 1.0, 3.0, 1.0]                                  # parameters
prob = ODEProblem(lotka_volterra!, ũ₀, timespan, p̃)       # define the problem
sol_lv = solve(prob)                                      # solve it
plot(sol_lv, label = ["Rabbits" "Foxes"])                 # plot solution

In [ ]:
@manipulate for α ∈ 0:0.1:5, β ∈ 0:0.1:5
    p̃ = [α β 3.0 1.0]
    prob = ODEProblem(lotka_volterra!, ũ₀, timespan, p̃)
    sol_lv = solve(prob)
    plot(sol_lv, label = ["Rabbits" "Foxes"])
end

https://github.com/SciML/DiffEqTutorials.jl

# 2. DynamicalSystems.jl

DynamicalSystems.jl is an award-winning library for nonlinear dynamics and chaos, part of [JuliaDynamics](https://juliadynamics.github.io/JuliaDynamics/)

<img src="map_of_ds.png">

## Using known system $f$


If you have a dynamical system where you know its rule $f$, there are a lot of things you can do with in DynamicalSystems.jl. But for all of them, step 1 is to make this rule $f$ into a `DynamicalSystem` instance. The process here is almost identical to DifferentialEquations.jl:

1. Make $f$ a Julia function (using the same syntax as in DifferentialEquations.jl).
2. Choose initial state and parameter container (again same as DiffEq).
3. Pass these arguments to the `ContinuousDynamicalSystem` constructor (or `DiscreteDynamicalSystem`, for discrete systems).

Here we already have `lorenz, u₀, p` defined, so the process would be just:

In [ ]:
lorenz_ds = ContinuousDynamicalSystem(lorenz, u₀, p)

Actually, DynamicalSystems.jl requires that we use StaticArrays when we provide an out-of-place equation of motion, which are similar to usual arrays but are stack-allocated and thus are more performant for small systems ( $\lesssim$ 100 ODEs/SDEs/DDEs/DAEs).

In [ ]:
function lorenz(u,p,t)
    x, y, z = u
    σ, ρ, β = p
    
    ẋ = σ*(y - x)
    ẏ = x*(ρ - z) - y
    ż = x*y - β*z
    
    return SVector(ẋ, ẏ, ż)
end
                                
static_u₀ = SVector(1.0, 5., 10.)

In [ ]:
lorenz_ds = ContinuousDynamicalSystem(lorenz, static_u₀, p)

Now we can use this `lorenz` object in various functions of the library. For example, to obtain the Lyapunov spectrum you use `lyapunovs`:

In [ ]:
lyapunovs(lorenz_ds, 10000) # 2nd argument is for how much time to evolve

Or if we want to get only the maximum Lyapunov exponent you can use the function `lyapunov` (which uses a different algorithm):

In [ ]:
lyapunov(lorenz_ds, 10000.0)

Obtaining a Poincare surface of section with the plane that e.g. the $y$ variable crosses zero:

In [ ]:
plane = (2, 0.0) # plane[1]: which variable, plane[2]: which value
psos = poincaresos(lorenz_ds, plane, 1000.0)

In [ ]:
scatter(psos[:, 1], psos[:, 3], label = false)
xlabel!("X")
ylabel!("Z")

## Nonlinear timeseries analysis



Many nonlinear timeseries analysis tools are included in DynamicalSystems.jl. such as Broomhead-King delay embeddings, time series prediction, calculating the generalized (Rényi) and permutation entropy, many different kinds attractor dimensions (like fractal dimension, Grassberger-Proccacia dimension, Kaplan-Yorke dimension), etc. 

Let's say that you have a nice trajectory sampled at discrete times (which is what the function `trajectory` does).

In [ ]:
tr = trajectory(lorenz_ds, 100.0; dt = 0.01)

Let's see an example of calculating the [generalized entropy](https://juliadynamics.github.io/DynamicalSystems.jl/dev/chaos/entropies/#Generalized-Entropy-1) of the attractor (which is calculated by partitioning the state space in boxes of size `ε`).

You can get this entropy (of order α) using the function `genentropy(α, ε, tr)`:

In [ ]:
H = genentropy(1, 0.1, tr)

In [ ]:
H = genentropy(1, 0.01, tr)

This quantity is interesting because it can be used to calculate the fractal dimension of the `tr` object. The way it works is simple: You look at how `H` changes for varying `ε`. If `D` is the fractal dimension of `tr`, then it typically holds that $H \sim ε^{-D}$.

In [ ]:
D_lorenz = generalized_dim(1, tr)

# 3. DataDrivenDiffEq.jl

[DataDrivenDiffEq.jl](https://datadriven.sciml.ai/dev/) is a package for estimating equation-free and equation-based models for discrete and continuous differential equations.

As opposed to parameter identification, these methods aim to find the governing equations of motion automatically from a given set of data. They do not require a known model as input. Instead, these methods take in data and return the differential equation model which generated the data.

## Sparse Identification of Nonlinear Dynamics (SINDy)

<img src="SINDy.png">

(extracted from the original [SINDy paper](https://www.pnas.org/content/113/15/3932))

Let's try it with the Lorenz system, that we've been working on. First let's get the time series data:

In [ ]:
X = Array(sol)

We generate the *ideal* derivative data.

In [ ]:
Ẋ = similar(X)
for (i, xi) in enumerate(eachcol(X))
    Ẋ[:,i] = lorenz(xi, p, 0.0)
end

To generate the symbolic equations, we need to define a ` Basis` over the variables `x y z`. In this example, we will use all monomials up to degree of 4 and their products:

In [ ]:
@variables x y z
u = Operation[x; y; z]
polys = Operation[]
for i ∈ 0:4
    for j ∈ 0:4
        for  k ∈ 0:4
            push!(polys, u[1]^i * u[2]^j * u[3]^k)
        end
    end
end
basis = Basis(polys, u)
print(basis)

But we could also add more functions to the basis:

In [ ]:
h = [cos(u[1]); sin(u[1]); u[1]*sin(u[2]); u[2]*cos(u[2])]

push!(basis, h)
print(basis)

Now we can select an optimizier, and run the SINDy algorithm

In [ ]:
opt = STRRidge(0.1)
Ψ = SInDy(X, Ẋ, basis, maxiter = 100, opt = opt, normalize = true)

In [ ]:
print_equations(Ψ)

In [ ]:
ps = parameters(Ψ)

Let's have a look at the ``L2``-Error and Akaikes Information Criterion of the result

In [ ]:
get_error(Ψ)

In [ ]:
get_aicc(Ψ)

To generate a numerical model usable in `DifferentialEquations`, we simply use the `ODESystem` function from `ModelingToolkit`.

In [ ]:
ps = parameters(Ψ)
sys = ODESystem(Ψ)
dudt = ODEFunction(sys)

prob = ODEProblem(dudt, u₀, tspan, ps)
solution = solve(prob, saveat = sol.t)

Let's have a look at the trajectory of $x(t)$.

In [ ]:
plot(solution, vars=1, label = "Estimation")
plot!(sol, vars = 1, label = "True")

Finally, let's investigate the error of the chaotic equations:

In [ ]:
ϵ = norm.(eachcol(solution .- sol))
plot(solution.t, ϵ .+ eps(), yaxis = :log, legend = false)
xlabel!("Time [s]")
ylabel!("L2 Error")

# 4. DiffEqFlux.jl

[DiffEqFlux.jl](https://diffeqflux.sciml.ai/) is another really exciting package from the [Scientific Machine Learning Ecosystem](https://sciml.ai). Besides implementing NeuralODEs/SDEs/DDEs, etc they provide tools for builing what they have called [Universal Differential Equations](https://arxiv.org/abs/2001.04385).

What if we only know some part of the differential equations?
Could we plug-in some neural networks to infer the missing terms?

In [ ]:
function lotka_volterra(u, p,t)
    🐇, 🦊 = u
    α, β, γ, δ = p

    d🐇 =  α*🐇 - β*🐇*🦊
    d🦊 = -γ*🦊 + δ*🐇*🦊
    return [d🐇, d🦊]
end

tspan = (0.0, 3.0)
u₀ = [0.4, 4.6]
p₀ = [1.3, 0.9, 1.8, 0.8]
prob = ODEProblem(lotka_volterra, u₀, tspan, p₀)
solution = solve(prob, Vern7(), abstol=1e-12, reltol=1e-12, saveat = 0.1)

scatter(solution, alpha = 0.25, label = false)
plot!(solution, alpha = 0.5, label = ["Rabbits" "Foxes"])

Create some noisy data from the true solution

In [ ]:
# Ideal data
tsdata = Array(solution)
# Add noise to the data
Random.seed!(333)
noisy_data = tsdata + 1e-5*randn(eltype(tsdata), size(tsdata))

plot(abs.(tsdata - noisy_data)')

Let's suppose that we actually don't know the underlying equations but we know or can guess some parts of it. We could use an artificial neural network to learn the missing terms:

$$
\begin{aligned}
\dot{x} &= \alpha x + NN_1(x, y) \\
\dot{y} &= - \gamma y + NN_2(x, y)\\
\end{aligned}
$$

In [ ]:
function lotka_volterra_univ(u,p,t)
    🐇, 🦊 = u
    α = p₀[1]
    γ = p₀[3]
    NN₁, NN₂ = ann(u, p)

    d🐇 =  α*🐇 + NN₁
    d🦊 = -γ*🦊 + NN₂

    return [d🐇, d🦊]
end

In [ ]:
# Define a neueral network for learning the parts
# of the equations that we don't know
ann = FastChain(FastDense(2, 32, tanh),
                FastDense(32, 32, tanh),
                FastDense(32, 2))

# The model weights are destructured into a vector of parameters
p = initial_params(ann)

prob_univ = ODEProblem(lotka_volterra_univ, u₀, tspan, p)
sol_univ = concrete_solve(prob_univ, Tsit5(), u₀, p, saveat = solution.t)

# plot the true solution and 
plot(solution)
plot!(sol_univ)

Define the prediction function and loss:

In [ ]:
function predict(θ)
    Array(concrete_solve(prob_univ, Vern7(), u₀, θ, saveat = solution.t,
                         abstol=1e-6, reltol=1e-6,
                         sensealg = InterpolatingAdjoint(autojacvec=ReverseDiffVJP())))
end

# No regularisation right now
function loss(θ)
    pred = predict(θ)
    sum(abs2, noisy_data .- pred), pred # + 1e-5*sum(sum.(abs, params(θ)))
end

loss(p)

A callback for printing the loss:

In [ ]:
const losses = []
callback(θ,l,pred) = begin
    push!(losses, l)
    if length(losses)%50==0
        println(losses[end])
    end
    false
end

We are ready to train it. First let's use ADAM for some iterations and then refine it with BFGS.

In [ ]:
res1 = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 100)

In [ ]:
res2 = DiffEqFlux.sciml_train(loss, res1.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 10000)

In [ ]:
# Plot the losses
plot(losses, yaxis = :log, xaxis = :log, xlabel = "Iterations", ylabel = "Loss")

In [ ]:
# Plot the data and the approximation
NNsolution = predict(res2.minimizer)
# Trained on noisy data vs real solution
plot(solution.t, tsdata')
plot!(solution.t, NNsolution')

In [ ]:
# Collect the state trajectory and the derivatives
X = noisy_data
# Ideal derivatives
DX = Array(solution(solution.t, Val{1})) #- [p[1]*(X[1,:])';  -p[4]*(X[2,:])']

prob_univ2 = ODEProblem(lotka_volterra_univ, u₀, tspan, res2.minimizer)
_sol = solve(prob_univ2, Tsit5())

DX_ = Array(_sol(solution.t, Val{1}))

# The learned derivatives
plot(DX')
plot!(DX_')

In [ ]:
# True missing terms
L = [-p₀[2]*(X[1,:].*X[2,:])'; p₀[4]*(X[1,:].*X[2,:])']

# Learned
L̂ = ann(X,res2.minimizer)

scatter(L')
plot!(L̂')

# scatter(abs.(L-L̂)', yaxis = :log)

Now we can extend the time span of the integration and see how it performs for long term predictions

In [ ]:
# Let's see some long term predictions
tspan_long = (0.0f0, 30.0f0)
prob = ODEProblem(lotka_volterra, u₀, tspan_long, p₀)
solution = solve(prob, Vern7(), abstol=1e-6, reltol=1e-6)

prob_univ = ODEProblem(lotka_volterra_univ, u₀, tspan_long, res2.minimizer)
sol_univ = solve(prob, Vern7(), abstol=1e-6, reltol=1e-6)



scatter(sol_univ, alpha = 0.25, label = ["Predicted Rabbits" "Predicted Foxes"])
plot!(solution, label = ["True Rabbits" "True Foxes"])
plot!([2.99,3.01],[0.0, maximum(hcat(Array(solution), Array(sol_univ)))],
      lw=2, color=:black, label = false)
annotate!([(1.5,9, text("Training \nData", 10, :center, :top, :black, "Helvetica"))])